In [17]:
import pandas as pd
import numpy as np
import string
import re
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfTransformer

In [18]:
!pip install -U textblob
!pip install nltk 
!pip install catboost

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

Requirement already up-to-date: textblob in /usr/local/lib/python3.6/dist-packages (0.15.3)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [19]:
train = pd.read_csv('train_features.csv')
test = pd.read_csv('test_features.csv')

text_train = pd.read_csv('train_text_limpio.csv')
text_test = pd.read_csv('test_text_limpio.csv')

In [20]:
train['text'] = text_train['text']
test['text'] = text_test['text']

In [21]:
#Pasos NLP:
#1: FILTRADO DE DATOS - parte en Limpieza de datos
#2: TOKENIZACION
#3: LEMATIZACION

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer() 

def nlp_text(text):
    #filtrado de signos de puntuacion
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    #tokenizacion
    tokens = re.split('\W+', text)
    #lematizacion
    text = [ps.stem(word) for word in tokens]
    return text

PARA TRAIN

In [29]:
train.text.fillna(' ', inplace=True)
test.text.fillna(' ', inplace=True)

In [23]:
tf_idf = TfidfVectorizer(analyzer=nlp_text)
tf_idf = tf_idf.fit_transform(train.text)
matrix_tf_idf = tf_idf.todense()
matrix_tf_idf 

matrix([[0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        ...,
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        [0.0584109, 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ]])

In [24]:
matrix_tf_idf.shape

(7613, 18564)

In [25]:
numeric_features = train[['cant_stop_words', 'prom_long_palabra', 'cant_puntuacion', 'cant_apariciones_keyword', 'cant_numeros', 'cant_mayusculas', 'cant_vocales']]
numeric_features

,cant_stop_words,prom_long_palabra,cant_puntuacion,cant_apariciones_keyword,cant_numeros,cant_mayusculas,cant_vocales
0,6,4.384615,1,61,0,10,25
1,0,4.571429,1,61,0,5,13
2,11,5.090909,3,61,0,2,45
3,1,7.125000,2,61,5,1,24
4,7,4.500000,2,61,0,3,25
...,...,...,...,...,...,...,...
7608,2,6.636364,5,61,1,7,20
7609,9,5.300000,5,61,0,6,39
7610,2,7.250000,11,61,9,10,12
7611,5,6.263158,5,61,0,4,49


In [26]:
matrix_features = np.hstack((matrix_tf_idf, numeric_features))

In [27]:
matriz_tfidf_train = pd.DataFrame(matrix_features)
matriz_tfidf_train.to_csv('features_train_tf_idf.csv', index=False)

PARA TEST

In [30]:
tf_idf = TfidfVectorizer(analyzer=nlp_text)
tf_idf_test = tf_idf.fit_transform(test.text)
matrix_tf_idf_test = tf_idf_test.todense()

numeric_features_test = test[['cant_stop_words', 'prom_long_palabra', 'cant_puntuacion', 'cant_apariciones_keyword', 'cant_numeros', 'cant_mayusculas', 'cant_vocales']]

matrix_features_test = np.hstack((matrix_tf_idf_test, numeric_features_test))

matriz_tfidf_test = pd.DataFrame(matrix_features_test)
matriz_tfidf_test.to_csv('features_test_tf_idf.csv', index=False)